In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models

In [43]:
import os
import random
import cv2
import matplotlib.pyplot as plt

In [44]:
df = pd.read_csv("full_df.csv")
df

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6387,4686,63,Male,4686_left.jpg,4686_right.jpg,severe nonproliferative retinopathy,proliferative diabetic retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4686_left.jpg
6388,4688,42,Male,4688_left.jpg,4688_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4688_left.jpg
6389,4689,54,Male,4689_left.jpg,4689_right.jpg,mild nonproliferative retinopathy,normal fundus,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4689_left.jpg
6390,4690,57,Male,4690_left.jpg,4690_right.jpg,mild nonproliferative retinopathy,mild nonproliferative retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4690_left.jpg


In [45]:
img_dir = r'C:\Users\KIIT\Documents\Deep Learning\preprocessed_images'

In [46]:
df = df.iloc[:,1:7]
df.head()

,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords
0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus
1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus
2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy
3,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy
4,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy


In [47]:
s1 = df['Left-Diagnostic Keywords']
s1.head()

0                                             cataract
1                                        normal fundus
2    laser spot，moderate non proliferative retinopathy
3                          macular epiretinal membrane
4               moderate non proliferative retinopathy
Name: Left-Diagnostic Keywords, dtype: object

In [48]:
s2 = df['Right-Diagnostic Keywords']
s2.head()

0                             normal fundus
1                             normal fundus
2    moderate non proliferative retinopathy
3         mild nonproliferative retinopathy
4    moderate non proliferative retinopathy
Name: Right-Diagnostic Keywords, dtype: object

In [49]:
s2.unique()

array(['normal fundus', 'moderate non proliferative retinopathy',
       'mild nonproliferative retinopathy',
       'moderate non proliferative retinopathy，epiretinal membrane',
       'vitreous degeneration',
       'moderate non proliferative retinopathy，hypertensive retinopathy',
       'pathological myopia', 'macular epiretinal membrane',
       'myelinated nerve fibers', 'drusen', 'epiretinal membrane',
       'hypertensive retinopathy', 'cataract',
       'moderate non proliferative retinopathy，abnormal pigment ',
       'macular epiretinal membrane，moderate non proliferative retinopathy',
       'dry age-related macular degeneration，glaucoma',
       'dry age-related macular degeneration',
       'moderate non proliferative retinopathy，laser spot',
       'epiretinal membrane over the macula',
       'moderate non proliferative retinopathy，pathological myopia',
       'macular epiretinal membrane，diabetic retinopathy',
       'diabetic retinopathy',
       'wet age-related macu

In [50]:
for i in range(6392):
    if 'age-related' in s1[i]:
        s1[i] = 'ARMD'
    if 'cataract' in s1[i]:
        s1[i] = 'Cataract'
    if 'pathological myopia' in s1[i]:
        s1[i] = 'Myopia'
    if 'normal' in s1[i]:
        s1[i] = 'Normal'
    if 'glaucoma' in s1[i]:
        s1[i] = 'Glaucoma'
    if 'hypertensive' in s1[i]:
        s1[i] = 'Hypertension'
    if 'diabetic' in s1[i]:
        s1[i] = 'Diabetic Retinopathy'

<ipython-input-50-5342e86ddb25>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1[i] = 'Cataract'
<ipython-input-50-5342e86ddb25>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1[i] = 'Normal'
<ipython-input-50-5342e86ddb25>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1[i] = 'Hypertension'
<ipython-input-50-5342e86ddb25>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the c

In [51]:
count=0
for i in range(6392):
    if 'Glaucoma' in s1[i]:
        count+=1
count

317

In [52]:
df_left_arm = df[df['Left-Diagnostic Keywords'] == 'ARMD']
df_left_arm.head()

,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords
34,35,Male,43_left.jpg,43_right.jpg,ARMD,dry age-related macular degeneration，glaucoma
39,69,Female,48_left.jpg,48_right.jpg,ARMD,dry age-related macular degeneration
44,65,Female,53_left.jpg,53_right.jpg,ARMD,dry age-related macular degeneration
46,62,Male,55_left.jpg,55_right.jpg,ARMD,dry age-related macular degeneration
85,73,Female,102_left.jpg,102_right.jpg,ARMD,dry age-related macular degeneration


In [53]:
len(df_left_arm)

266

In [54]:
s2

0                                           normal fundus
1                                           normal fundus
2                  moderate non proliferative retinopathy
3                       mild nonproliferative retinopathy
4                  moderate non proliferative retinopathy
                              ...                        
6387                   proliferative diabetic retinopathy
6388               moderate non proliferative retinopathy
6389                                        normal fundus
6390                    mild nonproliferative retinopathy
6391    hypertensive retinopathy，age-related macular d...
Name: Right-Diagnostic Keywords, Length: 6392, dtype: object

In [55]:
for i in range(6392):
    if 'age-related' in s2[i]:
        s2[i] = 'ARMD'
    if 'cataract' in s2[i]:
        s2[i] = 'Cataract'
    if 'pathological myopia' in s2[i]:
        s2[i] = 'Myopia'
    if 'normal' in s2[i]:
        s2[i] = 'Normal'
    if 'glaucoma' in s2[i]:
        s2[i] = 'Glaucoma'
    if 'hypertensive' in s2[i]:
        s2[i] = 'Hypertension'
    if 'diabetic' in s2[i]:
        s2[i] = 'Diabetic Retinopathy'

<ipython-input-55-a172d0bd1bc4>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s2[i] = 'Normal'
<ipython-input-55-a172d0bd1bc4>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s2[i] = 'Hypertension'
<ipython-input-55-a172d0bd1bc4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s2[i] = 'Myopia'
<ipython-input-55-a172d0bd1bc4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

In [56]:
df_rt_arm = df[df['Right-Diagnostic Keywords'] == 'ARMD']
df_rt_arm.head()

,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords
34,35,Male,43_left.jpg,43_right.jpg,ARMD,ARMD
39,69,Female,48_left.jpg,48_right.jpg,ARMD,ARMD
44,65,Female,53_left.jpg,53_right.jpg,ARMD,ARMD
46,62,Male,55_left.jpg,55_right.jpg,ARMD,ARMD
57,56,Female,71_left.jpg,71_right.jpg,Diabetic Retinopathy,ARMD


In [57]:
len(df_rt_arm)

285

In [58]:
df_arm_filenames = df_left_arm['Left-Fundus'].append(df_rt_arm['Right-Fundus'], ignore_index=True)
df_arm_filenames.head()

0     43_left.jpg
1     48_left.jpg
2     53_left.jpg
3     55_left.jpg
4    102_left.jpg
dtype: object

In [59]:
len(df_arm_filenames)

551

In [60]:
df_left_nor = df[df['Left-Diagnostic Keywords'] == 'Normal']
df_left_nor.head()

,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords
1,57,Male,1_left.jpg,1_right.jpg,Normal,Normal
7,59,Male,8_left.jpg,8_right.jpg,Normal,Normal
8,54,Male,9_left.jpg,9_right.jpg,Normal,vitreous degeneration
12,55,Male,14_left.jpg,14_right.jpg,Normal,macular epiretinal membrane
13,50,Male,15_left.jpg,15_right.jpg,Normal,myelinated nerve fibers


In [61]:
df_rt_nor = df[df['Right-Diagnostic Keywords'] == 'Normal']
df_rt_nor.head()

,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords
0,69,Female,0_left.jpg,0_right.jpg,Cataract,Normal
1,57,Male,1_left.jpg,1_right.jpg,Normal,Normal
7,59,Male,8_left.jpg,8_right.jpg,Normal,Normal
9,70,Male,10_left.jpg,10_right.jpg,epiretinal membrane,Normal
21,63,Female,26_left.jpg,26_right.jpg,moderate non proliferative retinopathy,Normal


In [62]:
df_nor_filenames = df_left_nor['Left-Fundus'].append(df_rt_nor['Right-Fundus'],ignore_index=True)
df_nor_filenames

0           1_left.jpg
1           8_left.jpg
2           9_left.jpg
3          14_left.jpg
4          15_left.jpg
             ...      
5678    4644_right.jpg
5679    4659_right.jpg
5680    4660_right.jpg
5681    4671_right.jpg
5682    4689_right.jpg
Length: 5683, dtype: object

In [63]:
len(df_nor_filenames)

5683

In [64]:
df_nor_filenames = df_nor_filenames.sample(600)
df_nor_filenames = df_nor_filenames.reset_index(drop=True)
df_nor_filenames.head()

0     2912_left.jpg
1    4123_right.jpg
2     264_right.jpg
3     3219_left.jpg
4    2341_right.jpg
dtype: object

In [65]:
len(df_nor_filenames)

600

In [66]:
df_left_cat = df[df['Left-Diagnostic Keywords'] == 'Cataract']
df_left_cat.head()

,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords
0,69,Female,0_left.jpg,0_right.jpg,Cataract,Normal
65,66,Male,81_left.jpg,81_right.jpg,Cataract,Cataract
86,55,Female,103_left.jpg,103_right.jpg,Cataract,Normal
100,59,Male,119_left.jpg,119_right.jpg,Cataract,drusen
218,60,Female,254_left.jpg,254_right.jpg,Cataract,macular epiretinal membrane


In [67]:
df_rt_cat = df[df['Right-Diagnostic Keywords'] == 'Cataract']
df_rt_cat.head()

,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords
20,75,Female,24_left.jpg,24_right.jpg,Normal,Cataract
65,66,Male,81_left.jpg,81_right.jpg,Cataract,Cataract
93,57,Female,112_left.jpg,112_right.jpg,Normal,Cataract
160,55,Female,188_left.jpg,188_right.jpg,Diabetic Retinopathy,Cataract
186,36,Female,218_left.jpg,218_right.jpg,refractive media opacity,Cataract


In [68]:
df_cat_filenames = df_left_cat['Left-Fundus'].append(df_rt_cat['Right-Fundus'],ignore_index=True)
df_cat_filenames

0          0_left.jpg
1         81_left.jpg
2        103_left.jpg
3        119_left.jpg
4        254_left.jpg
            ...      
589    2243_right.jpg
590    2246_right.jpg
591    2247_right.jpg
592    2248_right.jpg
593    2282_right.jpg
Length: 594, dtype: object

In [69]:
df_left_myo = df[df['Left-Diagnostic Keywords'] == 'Myopia']
df_left_myo.head()

,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords
11,60,Female,13_left.jpg,13_right.jpg,Myopia,Myopia
16,58,Male,18_left.jpg,18_right.jpg,Myopia,Myopia
29,68,Female,35_left.jpg,35_right.jpg,Myopia,Normal
88,57,Female,106_left.jpg,106_right.jpg,Myopia,Myopia
122,49,Female,144_left.jpg,144_right.jpg,Myopia,Normal


In [70]:
df_rt_myo = df[df['Right-Diagnostic Keywords'] == 'Myopia']
df_rt_myo.head()

,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords
11,60,Female,13_left.jpg,13_right.jpg,Myopia,Myopia
14,54,Female,16_left.jpg,16_right.jpg,Normal,Myopia
16,58,Male,18_left.jpg,18_right.jpg,Myopia,Myopia
37,66,Male,46_left.jpg,46_right.jpg,moderate non proliferative retinopathy,Myopia
45,66,Female,54_left.jpg,54_right.jpg,Normal,Myopia


In [71]:
df_myo_filenames = df_left_myo['Left-Fundus'].append(df_rt_myo['Right-Fundus'],ignore_index=True)
df_myo_filenames

0         13_left.jpg
1         18_left.jpg
2         35_left.jpg
3        106_left.jpg
4        144_left.jpg
            ...      
452    1681_right.jpg
453    1686_right.jpg
454    1694_right.jpg
455    1718_right.jpg
456    1971_right.jpg
Length: 457, dtype: object

In [72]:
df_left_gla = df[df['Left-Diagnostic Keywords'] == 'Glaucoma']
df_left_gla.head()

,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords
78,46,Male,95_left.jpg,95_right.jpg,Glaucoma,Hypertension
129,79,Male,153_left.jpg,153_right.jpg,Glaucoma,ARMD
141,71,Male,167_left.jpg,167_right.jpg,Glaucoma,Glaucoma
319,60,Female,365_left.jpg,365_right.jpg,Glaucoma,Glaucoma
510,54,Male,583_left.jpg,583_right.jpg,Glaucoma,Glaucoma


In [73]:
df_rt_gla = df[df['Right-Diagnostic Keywords'] == 'Glaucoma']
df_rt_gla.head()

,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords
141,71,Male,167_left.jpg,167_right.jpg,Glaucoma,Glaucoma
319,60,Female,365_left.jpg,365_right.jpg,Glaucoma,Glaucoma
510,54,Male,583_left.jpg,583_right.jpg,Glaucoma,Glaucoma
648,59,Male,746_left.jpg,746_right.jpg,Glaucoma,Glaucoma
804,65,Male,931_left.jpg,931_right.jpg,Glaucoma,Glaucoma


In [74]:
df_gla_filenames = df_left_gla['Left-Fundus'].append(df_rt_gla['Right-Fundus'],ignore_index=True)
df_gla_filenames

0         95_left.jpg
1        153_left.jpg
2        167_left.jpg
3        365_left.jpg
4        583_left.jpg
            ...      
581    1999_right.jpg
582    2030_right.jpg
583    2032_right.jpg
584    2048_right.jpg
585    2063_right.jpg
Length: 586, dtype: object

In [75]:
df_left_hyp = df[df['Left-Diagnostic Keywords'] == 'Hypertension']
df_left_hyp.head()

,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords
10,60,Female,11_left.jpg,11_right.jpg,Hypertension,Hypertension
19,47,Male,23_left.jpg,23_right.jpg,Hypertension,Hypertension
23,63,Female,28_left.jpg,28_right.jpg,Hypertension,Hypertension
26,64,Female,32_left.jpg,32_right.jpg,Hypertension,Hypertension
36,54,Male,45_left.jpg,45_right.jpg,Hypertension,Hypertension


In [76]:
df_rt_hyp = df[df['Right-Diagnostic Keywords'] == 'Hypertension']
df_rt_hyp.head()

,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords
10,60,Female,11_left.jpg,11_right.jpg,Hypertension,Hypertension
19,47,Male,23_left.jpg,23_right.jpg,Hypertension,Hypertension
23,63,Female,28_left.jpg,28_right.jpg,Hypertension,Hypertension
26,64,Female,32_left.jpg,32_right.jpg,Hypertension,Hypertension
36,54,Male,45_left.jpg,45_right.jpg,Hypertension,Hypertension


In [77]:
df_hyp_filenames = df_left_hyp['Left-Fundus'].append(df_rt_hyp['Right-Fundus'],ignore_index=True)
df_hyp_filenames

0         11_left.jpg
1         23_left.jpg
2         28_left.jpg
3         32_left.jpg
4         45_left.jpg
            ...      
346    4562_right.jpg
347    4579_right.jpg
348    4581_right.jpg
349    4591_right.jpg
350    4592_right.jpg
Length: 351, dtype: object

In [78]:
df_left_drp = df[df['Left-Diagnostic Keywords'] == 'Diabetic Retinopathy']
df_left_drp.head()

,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords
52,80,Male,64_left.jpg,64_right.jpg,Diabetic Retinopathy,Diabetic Retinopathy
55,56,Male,67_left.jpg,67_right.jpg,Diabetic Retinopathy,Diabetic Retinopathy
57,56,Female,71_left.jpg,71_right.jpg,Diabetic Retinopathy,ARMD
90,54,Female,108_left.jpg,108_right.jpg,Diabetic Retinopathy,Normal
160,55,Female,188_left.jpg,188_right.jpg,Diabetic Retinopathy,Cataract


In [79]:
df_rt_drp = df[df['Right-Diagnostic Keywords'] == 'Diabetic Retinopathy']
df_rt_drp.head()

,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords
52,80,Male,64_left.jpg,64_right.jpg,Diabetic Retinopathy,Diabetic Retinopathy
55,56,Male,67_left.jpg,67_right.jpg,Diabetic Retinopathy,Diabetic Retinopathy
201,54,Male,234_left.jpg,234_right.jpg,Diabetic Retinopathy,Diabetic Retinopathy
213,64,Male,249_left.jpg,249_right.jpg,moderate non proliferative retinopathy,Diabetic Retinopathy
231,49,Female,268_left.jpg,268_right.jpg,moderate non proliferative retinopathy，laser spot,Diabetic Retinopathy


In [80]:
df_drp_filenames = df_left_drp['Left-Fundus'].append(df_rt_drp['Right-Fundus'],ignore_index=True)
df_drp_filenames

0        64_left.jpg
1        67_left.jpg
2        71_left.jpg
3       108_left.jpg
4       188_left.jpg
           ...      
86    4131_right.jpg
87    4391_right.jpg
88    4624_right.jpg
89    4631_right.jpg
90    4686_right.jpg
Length: 91, dtype: object

In [83]:
df_nor_filenames = pd.DataFrame(df_nor_filenames, columns = ["filename"])
df_nor_filenames["label"] = "Normal"
df_nor_filenames.head()

,filename,label
0,2912_left.jpg,Normal
1,4123_right.jpg,Normal
2,264_right.jpg,Normal
3,3219_left.jpg,Normal
4,2341_right.jpg,Normal


In [84]:
df_myo_filenames = pd.DataFrame(df_myo_filenames, columns = ["filename"])
df_myo_filenames["label"] = "Myopia"
df_myo_filenames.head()

,filename,label
0,13_left.jpg,Myopia
1,18_left.jpg,Myopia
2,35_left.jpg,Myopia
3,106_left.jpg,Myopia
4,144_left.jpg,Myopia


In [85]:
df_cat_filenames = pd.DataFrame(df_cat_filenames, columns = ["filename"])
df_cat_filenames["label"] = "Cataract"
df_cat_filenames.head()

,filename,label
0,0_left.jpg,Cataract
1,81_left.jpg,Cataract
2,103_left.jpg,Cataract
3,119_left.jpg,Cataract
4,254_left.jpg,Cataract


In [86]:
df_gla_filenames = pd.DataFrame(df_gla_filenames, columns = ["filename"])
df_gla_filenames["label"] = "Glaucoma"
df_gla_filenames.head()

,filename,label
0,95_left.jpg,Glaucoma
1,153_left.jpg,Glaucoma
2,167_left.jpg,Glaucoma
3,365_left.jpg,Glaucoma
4,583_left.jpg,Glaucoma


In [87]:
df_drp_filenames = pd.DataFrame(df_drp_filenames, columns = ["filename"])
df_drp_filenames["label"] = "Diabetic_Retinopathy"
df_drp_filenames.head()

,filename,label
0,64_left.jpg,Diabetic_Retinopathy
1,67_left.jpg,Diabetic_Retinopathy
2,71_left.jpg,Diabetic_Retinopathy
3,108_left.jpg,Diabetic_Retinopathy
4,188_left.jpg,Diabetic_Retinopathy


In [88]:
df_hyp_filenames = pd.DataFrame(df_hyp_filenames, columns = ["filename"])
df_hyp_filenames["label"] = "Hypertension"
df_hyp_filenames.head()

,filename,label
0,11_left.jpg,Hypertension
1,23_left.jpg,Hypertension
2,28_left.jpg,Hypertension
3,32_left.jpg,Hypertension
4,45_left.jpg,Hypertension


In [89]:
df_arm_filenames = pd.DataFrame(df_arm_filenames, columns = ["filename"])
df_arm_filenames["label"] = "ARMD"
df_arm_filenames.head()

,filename,label
0,43_left.jpg,ARMD
1,48_left.jpg,ARMD
2,53_left.jpg,ARMD
3,55_left.jpg,ARMD
4,102_left.jpg,ARMD


In [90]:
df_combined = df_arm_filenames.append([df_nor_filenames,df_cat_filenames,df_drp_filenames,df_gla_filenames,df_hyp_filenames,df_myo_filenames],ignore_index = True)
df_combined

,filename,label
0,43_left.jpg,ARMD
1,48_left.jpg,ARMD
2,53_left.jpg,ARMD
3,55_left.jpg,ARMD
4,102_left.jpg,ARMD
...,...,...
3225,1681_right.jpg,Myopia
3226,1686_right.jpg,Myopia
3227,1694_right.jpg,Myopia
3228,1718_right.jpg,Myopia


In [91]:
df_combined = df_combined.sample(3230)
df_combined = df_combined.reset_index(drop=True)
df_combined

,filename,label
0,1850_right.jpg,ARMD
1,2211_right.jpg,Cataract
2,1613_right.jpg,Myopia
3,1474_left.jpg,Glaucoma
4,2225_left.jpg,Cataract
...,...,...
3225,1879_left.jpg,ARMD
3226,139_right.jpg,Hypertension
3227,345_right.jpg,Cataract
3228,2409_right.jpg,Normal


In [97]:
a  = np.array(df_combined.filename)

In [98]:
a.shape

(3230,)

In [99]:
paths = []
type(paths)

list

In [100]:
for i in range(3230):
    img = a[i]
    image = os.path.join(img_dir, img)
    paths.append(image)

In [101]:
paths

['C:\\Users\\KIIT\\Documents\\Deep Learning\\preprocessed_images\\1850_right.jpg',
 'C:\\Users\\KIIT\\Documents\\Deep Learning\\preprocessed_images\\2211_right.jpg',
 'C:\\Users\\KIIT\\Documents\\Deep Learning\\preprocessed_images\\1613_right.jpg',
 'C:\\Users\\KIIT\\Documents\\Deep Learning\\preprocessed_images\\1474_left.jpg',
 'C:\\Users\\KIIT\\Documents\\Deep Learning\\preprocessed_images\\2225_left.jpg',
 'C:\\Users\\KIIT\\Documents\\Deep Learning\\preprocessed_images\\1845_left.jpg',
 'C:\\Users\\KIIT\\Documents\\Deep Learning\\preprocessed_images\\1540_right.jpg',
 'C:\\Users\\KIIT\\Documents\\Deep Learning\\preprocessed_images\\264_right.jpg',
 'C:\\Users\\KIIT\\Documents\\Deep Learning\\preprocessed_images\\2127_right.jpg',
 'C:\\Users\\KIIT\\Documents\\Deep Learning\\preprocessed_images\\1243_right.jpg',
 'C:\\Users\\KIIT\\Documents\\Deep Learning\\preprocessed_images\\1644_right.jpg',
 'C:\\Users\\KIIT\\Documents\\Deep Learning\\preprocessed_images\\2243_right.jpg',
 'C:\\Us

In [102]:
len(paths)

3230

In [246]:
data = []

In [247]:
for i in range(3230):
    img = paths[i]
    image = cv2.imread(img)
    image = cv2.resize(image,(224,224))
    data.append(image)

In [248]:
len(data)

3230

In [249]:
data = np.array(data)

In [250]:
data = data/255

In [251]:
x = data
y = []
for i in df_combined.label:
    if(i=='Normal'):
        y.append(0)
    elif(i=='ARMD'):
        y.append(1)
    elif(i=='Cataract'):
        y.append(2)
    elif(i=='Glaucoma'):
        y.append(3)
    elif(i=='Myopia'):
        y.append(4)
    elif(i=='Diabetic_Retinopathy'):
        y.append(5)
    else:
        y.append(6)
y = np.array(y)

In [252]:
len(x)

3230

In [253]:
len(y)

3230

In [255]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=0.1)
x_val,x_test,y_val,y_test = train_test_split(x_val,y_val,test_size=0.5)

print(len(x_train))
print(len(x_val))
print(len(x_test))

2907
161
162


# MobileNet V2 Model

In [256]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [257]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [258]:
num_of_classes = 7

model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_of_classes)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 7)                 8967      
Total params: 2,266,951
Trainable params: 8,967
Non-trainable params: 2,257,984
_________________________________________________________________


In [259]:
model.compile(
  optimizer="nadam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

model.fit(x_train, y_train, epochs=15)

Epoch 1/15
91/91 [==============================] - 56s 536ms/step - loss: 1.2773 - accuracy: 0.5174
Epoch 2/15
91/91 [==============================] - 49s 537ms/step - loss: 0.9488 - accuracy: 0.6436
Epoch 3/15
91/91 [==============================] - 49s 539ms/step - loss: 0.8431 - accuracy: 0.6890
Epoch 4/15
91/91 [==============================] - 49s 536ms/step - loss: 0.7754 - accuracy: 0.7100
Epoch 5/15
91/91 [==============================] - 49s 541ms/step - loss: 0.7203 - accuracy: 0.7417
Epoch 6/15
91/91 [==============================] - 49s 540ms/step - loss: 0.6795 - accuracy: 0.7578
Epoch 7/15
91/91 [==============================] - 50s 545ms/step - loss: 0.6429 - accuracy: 0.7719
Epoch 8/15
91/91 [==============================] - 50s 544ms/step - loss: 0.6072 - accuracy: 0.7929
Epoch 9/15
91/91 [==============================] - 49s 541ms/step - loss: 0.5815 - accuracy: 0.8025
Epoch 10/15
91/91 [==============================] - 51s 557ms/step - loss: 0.5533 - accura

In [260]:
model.evaluate(x_train,y_train)

91/91 [==============================] - 55s 589ms/step - loss: 0.4279 - accuracy: 0.8731


[0.4278706908226013, 0.8730649948120117]

In [292]:
y_pre = model.predict(x_test)
y_pred = [np.argmax(i) for i in y_pre]
y_pred[:10]

[0, 2, 6, 1, 2, 6, 2, 1, 2, 0]

In [294]:
y_test[:10]

array([0, 2, 6, 1, 0, 6, 2, 1, 2, 0])

In [296]:
from sklearn.metrics import classification_report

print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.65      0.50      0.57        30
           1       0.81      0.93      0.86        27
           2       0.89      0.83      0.86        29
           3       0.73      0.85      0.79        26
           4       0.89      0.96      0.92        25
           5       0.50      1.00      0.67         2
           6       0.85      0.74      0.79        23

    accuracy                           0.80       162
   macro avg       0.76      0.83      0.78       162
weighted avg       0.80      0.80      0.79       162



### Resnet 50

In [303]:
num_classes = 7
model2 = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/tensorflow/resnet_50/classification/1",
                   trainable=False,input_shape=(224, 224, 3)),  
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [304]:
model2 = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_of_classes)
])

model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 8967      
Total params: 2,266,951
Trainable params: 8,967
Non-trainable params: 2,257,984
_________________________________________________________________


In [305]:
model2.compile(
  optimizer="nadam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

model2.fit(x_train, y_train, epochs=15)

Epoch 1/15
91/91 [==============================] - 46s 496ms/step - loss: 1.2605 - accuracy: 0.5280
Epoch 2/15
91/91 [==============================] - 49s 542ms/step - loss: 0.9472 - accuracy: 0.6495
Epoch 3/15
91/91 [==============================] - 51s 561ms/step - loss: 0.8423 - accuracy: 0.6883
Epoch 4/15
91/91 [==============================] - 51s 559ms/step - loss: 0.7732 - accuracy: 0.7155
Epoch 5/15
91/91 [==============================] - 49s 535ms/step - loss: 0.7148 - accuracy: 0.7403
Epoch 6/15
91/91 [==============================] - 50s 551ms/step - loss: 0.6727 - accuracy: 0.7609
Epoch 7/15
91/91 [==============================] - 52s 574ms/step - loss: 0.6355 - accuracy: 0.7843
Epoch 8/15
91/91 [==============================] - 56s 613ms/step - loss: 0.6013 - accuracy: 0.8001
Epoch 9/15
91/91 [==============================] - 52s 570ms/step - loss: 0.5751 - accuracy: 0.8077
Epoch 10/15
91/91 [==============================] - 51s 562ms/step - loss: 0.5494 - accura

In [306]:
model2.evaluate(x_test,y_test)

6/6 [==============================] - 3s 342ms/step - loss: 0.6128 - accuracy: 0.8395


[0.6127630472183228, 0.8395061492919922]

In [307]:
model2.evaluate(x_val,y_val)

6/6 [==============================] - 2s 341ms/step - loss: 0.5950 - accuracy: 0.7640


[0.5950422286987305, 0.7639751434326172]